In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [5]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

neoai_2025_underfitting_cv_path = kagglehub.competition_download('neoai-2025-underfitting-cv')

print('Data source import complete.')


100%|██████████| 448M/448M [00:08<00:00, 56.0MB/s]

Extracting files...


Data source import complete.


In [6]:
neoai_2025_underfitting_cv_path

'/root/.cache/kagglehub/competitions/neoai-2025-underfitting-cv'

In [7]:
import pandas as pd
import numpy as np
import os
import albumentations as A
from torch.utils.data import Dataset, DataLoader, SequentialSampler, RandomSampler
import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot
import timm
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import Dataset, DataLoader, SequentialSampler, RandomSampler
import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot
import timm
import torch.nn as nn
import torch.nn.functional as F
import torch
import random

## Legend
## Slava's glass. Pt. 1

Slava had a glass.
But not just any glass — his glass.
It wasn’t dishwasher-safe, it had a chip on the rim and a suspicious stain from the 9th grade — but it brought luck, prestige, and the kind of quiet power you feel when you slam it down after solving a problem no one else even attempted.

Enter Andrey.

— “Lend it to me for a day. I’ll make you two just like it. Better, even. With a non-slip base and maybe a gold rim.”

— “You're serious?”

— “I give you my programmer word.”

Slava trusted him.
Slava was a fool.

The next day:

— “Where’s my glass?”

— “Gone.”

— “Gone where?”

— “To a better place. It's in a safe now.”

— “You said you’d make two more!”

— “Oh, I lied. Let's just say... you lost more than just a glass.”

That’s when Slava realized — Andrey wasn’t just talking about a drinking vessel.
He meant his glass.

That night, Slava broke into Andrey’s apartment.
The safe gleamed in the dark like a physics teacher’s conscience during finals.

Suddenly, the screen lit up with a message:

❗ Want your glass back? Solve this:

You have a neural network that performs well on some classes, but struggles with others.
Luckily, you’ve been given extra data for those underperforming classes.

Your task:
— Fine-tune the model so it handles all classes effectively;
— And make sure it doesn’t forget what it already knows —
(catastrophic forgetting is your enemy).

Slava sat down.
Opened his laptop.
Pressed play on an old mp3 in his headphones.

Al Pacino’s voice came on, raspy and fired up:

“I don’t know what to say, really. Three minutes till the biggest battle of our professional lives all comes down to today...”

Slava took a breath.
— “Alright, glass. Let’s bring you home.”

## Overview

In this competition, you will need to retrain the neural network. The initial neural network that you have been given works well on one part (**GOOD CLASSES**) of the dataset (90 percent accuracy). But on the other part of the dataset (**BAD CLASSES**), the accuracy is about 30 percent. You need to achieve as much accuracy as possible on each part of the datasets. To do this, we will provide you with a small dataset for training (10 samples per class). However, it contains only bad classes. Your task is to retrain the neural network in order to improve the quality of the bad classes and keep the quality of the good classes the same.

## Metric

The harmonic mean between the accuracy of the good classes (**acc_good**) and the bad classes  (**acc_bad**).
$$ SCORE = \sqrt{(acc_{good} * acc_{bad})} $$,

## Restriction

For generating submission you **should use make_predict function**. You can only use data from the folder **train_images** for **finetuning network**.

## Data

* **test_images** - folder with images for creating submission file
* **train_images** - folder with images for finetuning network
* **model.pt** - initial weight of the pretrained network
* **sample_submission.csv** - example of submission file
* **Columns:**
* **id** - image filename in test_images folder
* **class** - class that you predict
* **train.csv** - dataset with classes for finetuning for **bad_classes**
* **Columns:**
* **path** - image filename in train_images folder
* **class** - class of image

Creating dataframes for training and inference

In [8]:
train = pd.read_csv('/root/.cache/kagglehub/competitions/neoai-2025-underfitting-cv/train.csv')
train['path'] = [f'/root/.cache/kagglehub/competitions/neoai-2025-underfitting-cv/train_images/{x}' for x in train['path']]

In [9]:
paths_list = []
main_path = '/root/.cache/kagglehub/competitions/neoai-2025-underfitting-cv/test_images'
for path in sorted(os.listdir(main_path)):
    paths_list += [f'{main_path}/{path}']

test = pd.DataFrame()
test['path'] = paths_list
test['class'] = 0

Create a model and dataset for training

In [35]:
class TrainDataset(Dataset):
    def __init__(self, path, target, transform):
        self.path = path
        self.target = target
        self.transform = transform
    def __len__(self):
        return len(self.target)

    def __getitem__(self, item):

        image = cv2.imread(self.path[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        target = self.target[item]
        image = self.transform(image=image)['image']
        image = image.astype(np.float32) / 255.0
        image = image - 0.5
        image = torch.from_numpy(image).permute(2, 0, 1)

        return image, target

class PetNet(nn.Module):
    def __init__( self, model_name, num_classes    ):
        super().__init__()
        self.model = timm.create_model(model_name, num_classes=num_classes)

    def forward(self, image):
        x = self.model(image)
        return x

def get_train_transforms(dim = 224):
    return A.Compose(
        [
            A.LongestMaxSize(max_size=dim, p=1.0),
            A.PadIfNeeded(dim, dim, p = 1.0),
            A.HorizontalFlip(p = 0.5),
        ]
  )

def get_valid_transforms(dim = 224):
    return A.Compose(
        [
            A.LongestMaxSize (max_size=dim, p=1.0),
            A.PadIfNeeded(dim, dim, p = 1.0),
        ]
  )

In [36]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

train loop

In [37]:
seed_everything(230)

batch_size = 32
epochs = 110
params_train = {'batch_size': batch_size, 'shuffle': True, 'drop_last': False, 'num_workers': 4}
params_valid = {'batch_size': batch_size, 'shuffle': False, 'drop_last': False, 'num_workers': 4}
lr = 2e-5
device = 'cuda'
dim = 224
N = 5

train_loader = DataLoader( TrainDataset( train['path'].tolist(), train['class'].tolist(), get_train_transforms(dim) ), **params_train)
valid_loader = DataLoader( TrainDataset( test['path'].tolist(), test['class'].tolist(), get_valid_transforms(dim) ), **params_valid)

model = PetNet("tiny_vit_5m_224.dist_in22k_ft_in1k", num_classes = 102)

model_dict = torch.load("/root/.cache/kagglehub/competitions/neoai-2025-underfitting-cv/model.pt", map_location='cuda', weights_only = False)
model.load_state_dict(model_dict, strict=False)

model = model.to(device)
model.train()

optimizer_parameters = model.parameters()
optimizer = AdamW(optimizer_parameters, lr=lr)
num_train_steps = int(len(train) / batch_size * epochs)
criterion = nn.CrossEntropyLoss(label_smoothing=0.)
scaler = torch.amp.GradScaler('cuda')
clip_grad_norm = 5

for epoch in range(epochs):
    len_dataloader = len(train_loader)
    average_loss = 0
    tk0 = tqdm(enumerate(train_loader), total = len_dataloader)
    for batch_number,  (inputs, labels)  in tk0:

        optimizer.zero_grad()
        inputs = inputs.cuda()
        labels = labels.cuda().long()

        with torch.amp.autocast('cuda'):
            y_preds  = model(inputs)
            loss = criterion(y_preds, labels)

        scaler.scale(loss).backward()

        if clip_grad_norm > 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        average_loss += loss.cpu().detach().numpy()
        tk0.set_postfix(loss=average_loss / (batch_number + 1), stage="train", epoch = epoch)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Inference function

**You cannot change this function.**

In [38]:
def make_predict(state_dict, valid_loader, name_csv = 'submission.csv', test_ids = [x.split('/')[-1] for x in test['path']]):
    preds = []
    len_loader = len(valid_loader)
    tk0 = tqdm(enumerate(valid_loader), total = len_loader)
    average_loss = 0
    model = timm.create_model( "tiny_vit_5m_224.dist_in22k_ft_in1k", num_classes=102)
    model.cuda().eval()
    model.load_state_dict(state_dict)

    with torch.no_grad():
        for batch_number,  (inputs, labels)  in tk0:
            inputs = inputs.cuda()
            labels = labels.cuda().long()

            with torch.amp.autocast('cuda'):
                y_preds  = model(inputs)

            preds += [y_preds.to('cpu').numpy()]

    preds = np.concatenate(preds)

    model.train()

    submission = pd.DataFrame()
    submission['id'] = test_ids
    submission['class'] = np.argmax(preds, 1)
    submission.to_csv(name_csv, index = None)

In [39]:
make_predict(model.model.state_dict(), valid_loader)

  0%|          | 0/160 [00:00<?, ?it/s]